# SQL Sat MN  Lab#2 - Code notebooks 2: DataWrangler

In [ ]:
# Run a SQL query to populate a dataframe from our PPP table (to keep the size down)
df = spark.sql("SELECT * FROM DuckLake.ppp_loan_details WHERE BorrowerState IN ('MN','ND','SD','IA','WI') LIMIT 600000")
display(df)

In [ ]:
df.count()

In [ ]:
df_pandas = df.select("*").toPandas()
df_pandas.head(50)

## My wrangled data is below    

In [ ]:
# Code generated by Data Wrangler for pandas DataFrame

def clean_data(df_pandas):
    # Change column type to datetime64[ns] for column: 'DateApproved'
    df_pandas = df_pandas.astype({'DateApproved': 'datetime64[ns]'})
    # Performed 1 aggregation grouped on columns: 'FranchiseName', 'BorrowerState'
    df_pandas = df_pandas.groupby(['FranchiseName', 'BorrowerState']).agg(ApprovedAmount_Sum=('ApprovedAmount', 'sum')).reset_index()
    # Sort by columns: 'FranchiseName' (ascending), 'BorrowerState' (ascending)
    df_pandas = df_pandas.sort_values(['FranchiseName', 'BorrowerState'])
    # Derive column 'ApprovedAmountTotal' from column: 'ApprovedAmount_Sum'
    # Transform based on the following examples:
    #    ApprovedAmount_Sum    Output
    # 1:            705374L => "705,374"
    df_pandas.insert(3, "ApprovedAmountTotal", df_pandas.apply(lambda row : f'{row["ApprovedAmount_Sum"]:01,.0f}', axis=1))
    # Drop column: 'ApprovedAmount_Sum'
    df_pandas = df_pandas.drop(columns=['ApprovedAmount_Sum'])
    return df_pandas

df_pandas_clean = clean_data(df_pandas.copy())
df_pandas_clean.head(1000)

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', None)

In [ ]:
df_pandas_clean.head(1000)

#### BONUS: Write your wrangled data back to a file and table

In [ ]:
# Pandas wrangled df to parquet file
df_pandas_clean.to_parquet("abfss://MNAdminDemo@onelake.dfs.fabric.microsoft.com/DuckLake.Lakehouse/Files/PPP/WrangledData.parquet")

In [ ]:
# Get the path by going to files - right click on the parquet file, Copy relative path

df_file = spark.read.parquet("Files/PPP/WrangledData.parquet")
df_file.write.mode("overwrite").format("delta").saveAsTable("ppp_wrangled")